In [1]:
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import time, re

webdriverpath = "/home/eesha/Softwares/chromedriver_linux64/chromedriver"
url = "https://www.youtube.com/watch?v=6PHsLkdE--g"

In [2]:
def save_comment(id_num, reply_to, title, channel, comment_text, comment_likes, comment_replies):
    comment_data = {}
    comment_data['id'] = id_num
    comment_data['reply to'] = reply_to
    comment_data['title'] = title
    comment_data['channel'] = channel
    comment_data['comment'] = comment_text
    comment_data['number of likes'] = comment_likes
    comment_data['number of replies'] = comment_replies
    return comment_data

In [5]:
def scraper(url):
    driver = webdriver.Chrome(webdriverpath) 
    driver.get(url)
    driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")

    title = driver.find_elements_by_xpath('//*[@id="container"]/h1/yt-formatted-string')[0].text
    channel = driver.find_element_by_xpath('//*[@id="text"]/a').text
    print("Video Title:", title)
    print("Channel:", channel)

    all_comments = []
    SCROLL_PAUSE_TIME = 2
    curr_num = 0

    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

        comment = driver.find_elements_by_xpath('//*[@id="content-text"]')
        like_count = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
        reply_count = driver.find_elements_by_xpath('//*[@id="replies"]/ytd-comment-replies-renderer')

        for c, lc, rc in zip(comment, like_count, reply_count):
            comment_text = c.text.replace('\n', ' ')
            comment_likes = lc.text
            comment_replies = rc.text.split(" ")[1].split(" ")[0]
            if 'K' in comment_likes:
                comment_likes = comment_likes.lower().split('k')[0]
                comment_likes = comment_likes + '000'

            if comment_replies.isnumeric() == False:
                comment_replies = '0'
                comment_data = save_comment(curr_num, 'None', title, channel, comment_text, comment_likes, comment_replies)
                all_comments.append(comment_data)
                curr_num += 1
            else:
                comment_data = save_comment(curr_num, 'None', title, channel, comment_text, comment_likes, comment_replies)
                all_comments.append(comment_data)
                parent_num = curr_num
                curr_num += 1

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
           break
        last_height = new_height

    return all_comments

In [6]:
video_comments = scraper(url)
print(video_comments)

Video Title: Amit Shah और Asaduddin Owaisi ने Lok Sabha में Citizenship Amendment Bill पर क्या कहा? (BBC Hindi)
Channel: BBC News Hindi


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

